In [1]:
import tensorflow as tf
import datetime
from tensorboard.plugins.hparams import api as hp

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [3]:
model = create_model()
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [4]:
log_dir="/Users/watcharinsarachai/Documents/_trained_models/test/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)

In [5]:
hparams_callback = hp.KerasCallback(log_dir, {
    'num_relu_units': 512,
    'dropout': 0.2
})

In [6]:
model.fit(
    x=x_train, 
    y=y_train, 
    epochs=5, 
    validation_data=(x_test, y_test), 
    callbacks=[tensorboard_callback, hparams_callback])

Epoch 1/5
1875/1875 [==============================] - 23s 12ms/step - loss: 0.2187 - accuracy: 0.9352 - val_loss: 0.1042 - val_accuracy: 0.9684
Epoch 2/5
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0961 - accuracy: 0.9707 - val_loss: 0.0714 - val_accuracy: 0.9777
Epoch 3/5
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0672 - accuracy: 0.9786 - val_loss: 0.0705 - val_accuracy: 0.9788
Epoch 4/5
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0521 - accuracy: 0.9829 - val_loss: 0.0631 - val_accuracy: 0.9802
Epoch 5/5
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0420 - accuracy: 0.9862 - val_loss: 0.0839 - val_accuracy: 0.9742
